<a href="https://colab.research.google.com/github/Storm00212/Livestock-ml-model/blob/main/Dairy_ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()  # Select your dairy_multi_disease_dataset.csv file

Saving dairy_multi_disease_dataset.csv to dairy_multi_disease_dataset.csv


In [2]:
import pandas as pd
import io

# Replace 'dairy_multi_disease_dataset.csv' with the exact filename if different
df = pd.read_csv(io.BytesIO(uploaded['dairy_multi_disease_dataset.csv']))

print("Dataset shape:", df.shape)
print("\nDisease label distribution:")
print(df['disease_label'].value_counts().sort_index())

print("\nFirst few rows:")
df.head()

Dataset shape: (120000, 13)

Disease label distribution:
disease_label
0    84011
1    10754
3        1
4    25234
Name: count, dtype: int64

First few rows:


,cow_id,day,ambient_temp,humidity,THI,body_temp,activity_index,feed_intake_kg,water_intake_l,milk_yield_l,milk_conductivity,scc_index,disease_label
0,1,1,27.490142,62.813497,76.614490,38.682953,1.011676,16.994441,88.262357,19.926198,5.167765,206.521450,0
1,1,2,25.585207,68.829386,74.566875,38.913109,0.984395,17.885238,68.898064,25.355140,5.846647,139.482995,0
2,1,3,27.943066,72.351420,78.553053,38.601165,0.971958,20.456203,62.726561,27.301111,5.598885,166.835388,4
3,1,4,30.569090,53.166354,79.451787,38.288539,0.790478,17.000584,66.393282,21.950857,5.458634,172.899109,4
4,1,5,25.297540,66.007390,73.831214,38.952856,0.940602,18.787450,84.397106,19.484860,5.261041,135.665743,0


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

# Features (clinically relevant ones - excluding cow_id and day for modeling)
features = ['ambient_temp', 'humidity', 'THI', 'body_temp', 'activity_index',
            'feed_intake_kg', 'water_intake_l', 'milk_yield_l',
            'milk_conductivity', 'scc_index']

# Filter out disease_label with only one sample to avoid stratification error
df_filtered = df[df['disease_label'] != 3].copy()

X = df_filtered[features]
y = df_filtered['disease_label']

# Handle class imbalance with class_weight='balanced'
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Training samples:", X_train.shape[0])
print("Test samples:", X_test.shape[0])

Training samples: 95999
Test samples: 24000
